In [1]:
# currently running "attempt 2"
# may still need to adapt the output of orthomosaics...
# might just have been (for the attempt 6) that I needed to read in the RGB and the MS seperately.
#####################################
# user defined stuff.
#####################################
import os

# Set license path
os.environ["agisoft_LICENSE"] = r"C:\ProgramData\Agisoft\Licensing\licenses\metashape-pro2.lic"

# Load metadata
metadata_path = r"\\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\PPKCorrectedFlightData\ProcessedRPAMetadata.csv"
output_path = r"D:\MetashapeTemp"
panel_cal_path = r"\\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Workflows\RP06-2202453-OB.csv"  # CSV from MicaSense

# Where to send the orthomosaic outputs
ortho_outputs = r"\\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Orthomosaics"


In [2]:
# attempt # 2 - one chunk...
# raw rgb
# calibrated MS
# no calibrated rgb camera
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]


def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path, ImageTypes = ["RGB","MS"]):
    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')
    chunk = doc.addChunk()
    chunk.label = "ALL"

    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to your TIFFs
    #task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)

    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".tif", ".tiff"])  # list of full file paths to your TIFFs
    task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)
    
    doc.save()
    print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Align images
    ######################################
    # add location accuracies.
    chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
    chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
        
    chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
    doc.save()
    
    chunk.alignCameras()
    doc.save()
    
    print("All images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Optimise alignment
    ######################################
    chunk.optimizeCameras(
        fit_f=True,
        fit_cx=True, fit_cy=True,
        fit_b1=False, fit_b2=False,
        fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
        fit_p1=True, fit_p2=True,
        fit_corrections=False,
        adaptive_fitting=False,
        tiepoint_covariance=False
    )
    print("Images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    # create different chunks based on image type
    for ImageType in ImageTypes:
        
        # Duplicate the main chunk
        dup_chunk = chunk.copy()
        dup_chunk.label = ImageType
        doc.chunks.append(dup_chunk)

        # remove the images from a given chunk
        if ImageType == "MS":
            # Remove RGB cameras from the MS chunk
            rgb_cams = [cam for cam in dup_chunk.cameras if "_D" in cam.label]
            dup_chunk.remove(rgb_cams)
        else:
            # Remove MS cameras from the RGB chunk
            ms_cams = [cam for cam in dup_chunk.cameras if "MS" in cam.label]
            dup_chunk.remove(ms_cams)

        doc.save()
        print(ImageType+ "Images copied to new chunk "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

        #####################################
        # Multispectral calibration
        #####################################
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
            task.apply(dup_chunk)
        
            dup_chunk.locateReflectancePanels()
            dup_chunk.loadReflectancePanelCalibration(panel_cal_path)
            # Set calibration task
            calib_task = Metashape.Tasks.CalibrateReflectance()
            calib_task.use_reflectance_panels = True
            calib_task.use_sun_sensor = False
            calib_task.apply(dup_chunk)
            print("multispectral images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
    
        ###############################################
        # Building models and orthos
        ###############################################
        dup_chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
        doc.save()
        
        dup_chunk.buildModel(source_data = Metashape.DepthMapsData)
        doc.save()
        
        dup_chunk.buildUV(page_count = 2, texture_size = 4096)
        doc.save()
        
        dup_chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
        doc.save()
        
        has_transform = dup_chunk.transform.scale and dup_chunk.transform.rotation and dup_chunk.transform.translation
        
        if has_transform:
            dup_chunk.buildPointCloud()
            doc.save()
        
            dup_chunk.buildDem(source_data=Metashape.PointCloudData)
            doc.save()
        
            dup_chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
            doc.save()
        
            print(ImageType + " ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        else:
            print("error: transform not available")
            
        # export results
        dup_chunk.exportReport(output_folder + '/report'+ImageType+'.pdf')
        
        if dup_chunk.model:
            dup_chunk.exportModel(output_folder + '/model'+ImageType+'.obj')
        
        if dup_chunk.point_cloud:
            dup_chunk.exportPointCloud(output_folder + '/point_cloud'+ImageType+'.las', source_data = Metashape.PointCloudData)
        
        if dup_chunk.orthomosaic:
            dup_chunk.exportRaster(output_folder + '/orthomosaic'+ImageType+'.tif', source_data = Metashape.OrthomosaicData)
        
        if dup_chunk.elevation and ImageType=="RGB":
            dup_chunk.exportRaster(output_folder + '/dsm'+ImageType+'.tif', source_data = Metashape.ElevationData)
        
            ground_task = Metashape.Tasks.ClassifyGroundPoints()
            ground_task.apply(dup_chunk)
            dup_chunk.buildDem(source_data=Metashape.PointCloudData, 
                       interpolation=Metashape.EnabledInterpolation, 
                       classes=[2])  # class 2 = ground points
            
            if dup_chunk.elevation:
                dup_chunk.exportRaster(output_folder + '/dtm'+ImageType+'.tif', source_data = Metashape.ElevationData)
                print(ImageType + " DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()
    

In [4]:
#####################################
# Apply function using loop
#####################################

merged_df = pd.read_csv(metadata_path)

for i in range(len(merged_df)):
    row = merged_df.iloc[i]
    
    # Resolve image folder paths (can be multiple, separated by ;)
    image_folders = row["ExifCorrectedImagePath"].split(';')
    flight_code = row["Flight code"]
    calib_before = row["Calibration_img_before"]
    calib_after = row["Calibration_img_after"]

    ####################################################################################
    # Create temporary output directory for this flight
    ####################################################################################
    project_folder = os.path.normpath(os.path.join(output_path, flight_code))
    temp_image_dir = os.path.join(project_folder, "MergedImages")
    temp_cal_dir = os.path.join(project_folder, "CalImages")

    
    final_output_path = os.path.join(ortho_outputs, flight_code)
    if os.path.isdir(final_output_path):
        print(f"folder already exists!: {final_output_path} manually delete then re-run.")
        continue

    # if it doesn't exist start moving things over to D drive.
    print(f"📁 Merging images to: {temp_image_dir}")
    
    os.makedirs(temp_image_dir, exist_ok=True)
    os.makedirs(temp_cal_dir, exist_ok=True)
    
    valid_exts = [".jpg", ".jpeg", ".tif", ".tiff"]
    for folder in image_folders:
        exif_dir = Path(folder) / "EXIF_images"  # explicitly go into subfolder
        if not exif_dir.exists():
            print(f"⚠️ Missing EXIF_images folder: {exif_dir}")
            continue
    
        for file in exif_dir.iterdir():
            if file.suffix.lower() in valid_exts and file.is_file():
                shutil.copy(file, temp_image_dir)
    
    ####################################################################################
    # Copy calibration images (handles 0–2 folders)
    ####################################################################################
    calib_dirs_raw = [calib_before, calib_after]           # whatever came from the CSV
    calib_dirs = [str(p) for p in calib_dirs_raw if pd.notna(p) and str(p).strip() != ""]
    calib_images = []
    
    if not calib_dirs:
        print("⚠️  No calibration folders listed for this flight.")
    else:
        print(f"🔎 Scanning {len(calib_dirs)} calibration folder(s)…")
    
    for calib_dir in calib_dirs:
        calib_path = Path(calib_dir)
        if calib_path.exists() and calib_path.is_dir():
            for file in calib_path.iterdir():
                if file.suffix.lower() in valid_exts and file.is_file():
                    dest = shutil.copy(file, temp_cal_dir)
                    calib_images.append(dest)
        else:
            print(f"⚠️  Calibration folder missing or invalid: {calib_path}")
    
    print(f"✅ Copied {len(calib_images)} calibration image(s) to {temp_cal_dir}")

    ##########################################
    # Pass outputs to metashape
    ##########################################
    ProcessImagesMetashape(temp_image_dir,
                           project_folder,
                           temp_cal_dir,
                           panel_cal_path)
    
    #########################################################
    # Move outputs to final destination and delete temp dir
    #########################################################
    # Delete temporary folders
    try:
        shutil.rmtree(os.path.join(project_folder, "MergedImages"))
        shutil.rmtree(os.path.join(project_folder, "CalImages"))
        print("🗑️ Temporary folders deleted.")
    except Exception as e:
        print(f"⚠️ Error deleting temporary folders: {e}")

    # Move outputs and delete temp root
    try:
        shutil.move(project_folder, final_output_path)
        print(f"📦 Project moved to final location: {final_output_path}")

    except Exception as e:
        print(f"⚠️ Error moving project folder: {e}")


folder already exists!: \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Orthomosaics\CR-F05-60m-20250509 manually delete then re-run.
folder already exists!: \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Orthomosaics\CR-F04-40m-20250509 manually delete then re-run.
folder already exists!: \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Orthomosaics\CR-F02-60m-20250509 manually delete then re-run.
folder already exists!: \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Orthomosaics\CR-F01-40m-20250509 manually delete then re-run.
folder already exists!: \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB\02_FieldData\00_NEW STRUCTURE\08_RPAData\Orthomosaics\MS-F11-60m-20250512 manually delete then re-run.
folder already exists!: \\nexus\csiro\HB\ECE\PASSIFLORA\ecology\08_Phase2_GBINCB

In [ ]:
#####################################
# Apply function to one set... loop
#####################################

merged_df = pd.read_csv(metadata_path)

i=2
row = merged_df.iloc[i]

# Resolve image folder paths (can be multiple, separated by ;)
image_folders = row["ExifCorrectedImagePath"].split(';')
flight_code = row["Flight code"]
calib_before = row["Calibration_img_before"]
calib_after = row["Calibration_img_after"]

####################################################################################
# Create temporary output directory for this flight
####################################################################################
project_folder = os.path.normpath(os.path.join(output_path, flight_code))
temp_image_dir = os.path.join(project_folder, "MergedImages")
temp_cal_dir = os.path.join(project_folder, "CalImages")

##########################################
# Pass outputs to metashape
##########################################
ProcessImagesMetashape(temp_image_dir,
                       project_folder,
                       temp_cal_dir,
                       panel_cal_path)

In [ ]:
# below chunks are alternative processing pathways that did not work....

In [ ]:
# Attempt 1 - 2 chunks for MS and RGB
# atm this is as good as it gets.
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]


def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path, ImageTypes = ["RGB","MS"]):
    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')

    for ImageType in ImageTypes:
        
        chunk = doc.addChunk()
        chunk.label = ImageType  # Name chunk after image type
        
        # set up photo read in.
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
        else:
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to jpgs
        
        task.load_reference = True
        task.load_xmp_accuracy = True
        task.load_xmp_orientation = True
        task.load_xmp_antenna = True
        task.load_xmp_calibration = True
        task.apply(chunk)
        
        doc.save()
        print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
        #####################################
        # Multispectral calibration
        #####################################
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
            task.apply(chunk)
            
            # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
            chunk.locateReflectancePanels()
            chunk.loadReflectancePanelCalibration(panel_cal_path)
            # Set calibration task
            calib_task = Metashape.Tasks.CalibrateReflectance()
            calib_task.use_reflectance_panels = True
            calib_task.use_sun_sensor = False
            #calib_task.panel_images = calib_cameras
            calib_task.apply(chunk)
            print("multispectral images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
            
        ######################################
        # Align images
        ######################################
        # add location accuracies.
        chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)

        if ImageType == "RGB":
            chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
        

            
        chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
        doc.save()
        
        chunk.alignCameras()
        doc.save()
        
        print(ImageType + " images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
        ######################################
        # Optimise alignment
        ######################################
        chunk.optimizeCameras(
            fit_f=True,
            fit_cx=True, fit_cy=True,
            fit_b1=False, fit_b2=False,
            fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
            fit_p1=True, fit_p2=True,
            fit_corrections=False,
            adaptive_fitting=False,
            tiepoint_covariance=False
        )
        print(ImageType + " images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

        if ImageType == "MS":
            # After aligning and optimizing MS chunk — now align to RGB
            try:
                chunk_labels = {chunk.label: chunk for chunk in doc.chunks}
                ms_chunk = chunk_labels.get("MS")
                rgb_chunk = chunk_labels.get("RGB")
                
                if ms_chunk and rgb_chunk:
                    doc.alignChunks(
                        chunks=[ms_chunk],
                        reference=rgb_chunk,
                        method=Metashape.PointBasedAlignment,
                        fit_scale=False,
                        downscale=1,
                        generic_preselection=True,
                        keypoint_limit=40000
                    )
                    print("✅ MS chunk aligned to RGB chunk (before orthos).")
                    doc.save()
                else:
                    print("⚠️ Cannot align chunks: RGB or MS chunk not found.")
            except Exception as e:
                print(f"⚠️ Chunk alignment failed: {e}")

        
        ###############################################
        # Building models and orthos
        ###############################################
        chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
        doc.save()
        
        chunk.buildModel(source_data = Metashape.DepthMapsData)
        doc.save()
        
        chunk.buildUV(page_count = 2, texture_size = 4096)
        doc.save()
        
        chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
        doc.save()
        
        has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
        
        if has_transform:
            chunk.buildPointCloud()
            doc.save()
        
            chunk.buildDem(source_data=Metashape.PointCloudData)
            doc.save()
        
            chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
            doc.save()
        
            print(ImageType + " ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        else:
            print("error: transform not available")
        # export results
        chunk.exportReport(output_folder + '/report'+ImageType+'.pdf')
        
        if chunk.model:
            chunk.exportModel(output_folder + '/model'+ImageType+'.obj')
        
        if chunk.point_cloud:
            chunk.exportPointCloud(output_folder + '/point_cloud'+ImageType+'.las', source_data = Metashape.PointCloudData)
        
        if chunk.elevation:
            chunk.exportRaster(output_folder + '/dsm'+ImageType+'.tif', source_data = Metashape.ElevationData)
        
        if chunk.orthomosaic:
            chunk.exportRaster(output_folder + '/orthomosaic'+ImageType+'.tif', source_data = Metashape.OrthomosaicData)
        
        
        ground_task = Metashape.Tasks.ClassifyGroundPoints()
        ground_task.apply(chunk)
        chunk.buildDem(source_data=Metashape.PointCloudData, 
                   interpolation=Metashape.EnabledInterpolation, 
                   classes=[2])  # class 2 = ground points
        
        
        if chunk.elevation:
            chunk.exportRaster(output_folder + '/dtm'+ImageType+'.tif', source_data = Metashape.ElevationData)
            print(ImageType + " DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()

In [ ]:
# Attempt 6 2 chunk for RGB and MS - MS also contains corrected RGB bands...
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]
        
def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path):

    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')

    for ImageType in ImageTypes:
        
        chunk = doc.addChunk()
        chunk.label = ImageType  # Name chunk after image type
        
        # set up photo read in.
        if ImageType == "RGB":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to jpgs
            task.load_reference = True
            task.load_xmp_accuracy = True
            task.load_xmp_orientation = True
            task.load_xmp_antenna = True
            task.load_xmp_calibration = True
            task.apply(chunk)
        
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to your TIFFs
            task.load_reference = True
            task.load_xmp_accuracy = True
            task.load_xmp_orientation = True
            task.load_xmp_antenna = True
            task.load_xmp_calibration = True
            task.apply(chunk)

            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
            task.load_reference = True
            task.load_xmp_accuracy = True
            task.load_xmp_orientation = True
            task.load_xmp_antenna = True
            task.load_xmp_calibration = True
            task.apply(chunk)
        
        # add location accuracies.
        chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
        chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
        
        doc.save()
        print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
        #####################################
        # Multispectral calibration
        #####################################
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
            task.apply(chunk)
            
            # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
            chunk.locateReflectancePanels()
            chunk.loadReflectancePanelCalibration(panel_cal_path)
            # Set calibration task
            calib_task = Metashape.Tasks.CalibrateReflectance()
            calib_task.use_reflectance_panels = True
            calib_task.use_sun_sensor = False
            #calib_task.panel_images = calib_cameras
            calib_task.apply(chunk)
            print(ImageType+" images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
            
        ######################################
        # Align images
        ######################################
        chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
        doc.save()
        
        chunk.alignCameras()
        doc.save()
        
        print(ImageType+" images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
        ######################################
        # Optimise alignment
        ######################################
        chunk.optimizeCameras(
            fit_f=True,
            fit_cx=True, fit_cy=True,
            fit_b1=False, fit_b2=False,
            fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
            fit_p1=True, fit_p2=True,
            fit_corrections=False,
            adaptive_fitting=False,
            tiepoint_covariance=False
        )
        print(ImageType+" images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
        # After aligning and optimizing MS chunk — now align to RGB
        try:
            chunk_labels = {chunk.label: i for i, chunk in enumerate(doc.chunks)}
            ms_index = chunk_labels.get("MS")
            rgb_index = chunk_labels.get("RGB")    
        
            if ms_index is not None and rgb_index is not None:
                task = Metashape.Tasks.AlignChunks()
                task.chunks = [ms_index]         # Chunks to align
                task.reference = rgb_index       # Reference chunk index
                task.method = 2                  # 0 = point based, 1 = marker, 2 = camera
                task.fit_scale = False
                task.downscale = 1               # High accuracy (1 = High, 0 = Highest)
                task.generic_preselection = True
                task.keypoint_limit = 40000
                task.apply(doc)
        
                print("✅ MS chunk aligned to RGB chunk (before orthos).")
                doc.save()
            else:
                print("⚠️ Cannot align chunks: RGB or MS chunk not found.")
        except Exception as e:
            print(f"⚠️ Chunk alignment failed: {e}")
            return break
        
        ###############################################
        # Building models and orthos
        ###############################################
        chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
        doc.save()
        
        chunk.buildModel(source_data = Metashape.DepthMapsData)
        doc.save()
        
        chunk.buildUV(page_count = 2, texture_size = 4096)
        doc.save()
        
        chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
        doc.save()
        
        has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
        
        if has_transform:
            chunk.buildPointCloud()
            doc.save()
        
            chunk.buildDem(source_data=Metashape.PointCloudData)
            doc.save()
        
            chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
            doc.save()
        
            print(ImageType+" ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        else:
            print("error: transform not available")
    
        ###################################
        # export results
        ###################################
        
        chunk.exportReport(output_folder + '/'+ImageType+'report'+'.pdf')
        
        if chunk.model:
            chunk.exportModel(output_folder + '/'+ImageType+'model'+'.obj')
        
        if chunk.point_cloud:
            chunk.exportPointCloud(output_folder + '/'+ImageType+'point_cloud'+'.las', source_data = Metashape.PointCloudData)
    
        if chunk.orthomosaic and ImageType == "RGB":
            chunk.exportRaster(output_folder + '/'+ImageType+'orthomosaic'+'.tif', source_data = Metashape.OrthomosaicData)
        
        if chunk.orthomosaic and ImageType == "MS":
            task = Metashape.Tasks.ExportRaster()
            task.source_data = Metashape.OrthomosaicData
            task.path = os.path.join(output_folder, f"{ImageType}_orthomosaic.tif")
            task.raster_transform = Metashape.RasterTransformType.RasterTransformNone  # preserve raw bands
            task.save_alpha = False
            task.image_format = Metashape.ImageFormat.ImageFormatTIFF
            task.clip_to_boundary = False  # optional, depending on whether you use shapes
            task.apply(chunk)
            
            for i in range(chunk.orthomosaic.bandCount()):
                chunk.exportRaster(
                    path=os.path.join(output_folder, f"MS_band_{i+1}.tif"),
                    source_data=Metashape.OrthomosaicData,
                    raster_transform=Metashape.RasterTransformType.RasterTransformNone,
                    bands=[i]  # zero-based index
                )
        
        if chunk.elevation and ImageType == "RGB":
            # export DSM
            chunk.exportRaster(output_folder + '/dsm'+'.tif', source_data = Metashape.ElevationData)
            
            # classify DTM
            ground_task = Metashape.Tasks.ClassifyGroundPoints()
            ground_task.apply(chunk)
            chunk.buildDem(source_data=Metashape.PointCloudData, 
                       interpolation=Metashape.EnabledInterpolation, 
                       classes=[2])  # class 2 = ground points
            
            # export DTM       
            chunk.exportRaster(output_folder + '/dtm'+'.tif', source_data = Metashape.ElevationData)
            print(" DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
            
    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()

In [ ]:
# attempt # 3 - one chunk as per MICASENSE...
# i think this failed...

#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]


def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path, ImageTypes = ["RGB","MS"]):
    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')
    chunk = doc.addChunk()
    chunk.label = "ALL"

    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to your TIFFs
    #task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)

    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".tif", ".tiff"])  # list of full file paths to your TIFFs
    task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)
    
    doc.save()
    print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Align images
    ######################################
    # add location accuracies.
    chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
    chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
        
    chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
    doc.save()
    
    chunk.alignCameras()
    doc.save()
    
    print("All images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Optimise alignment
    ######################################
    chunk.optimizeCameras(
        fit_f=True,
        fit_cx=True, fit_cy=True,
        fit_b1=False, fit_b2=False,
        fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
        fit_p1=True, fit_p2=True,
        fit_corrections=False,
        adaptive_fitting=False,
        tiepoint_covariance=False
    )
    print("Images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

                                                                
    ###############################################
    # Building models and orthos
    ###############################################
    chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
    doc.save()
    
    chunk.buildModel(source_data = Metashape.DepthMapsData)
    doc.save()
    
    chunk.buildUV(page_count = 2, texture_size = 4096)
    doc.save()
    
    chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
    doc.save()
    
    has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
            
    if has_transform:
        chunk.buildPointCloud()
        doc.save()
        
        chunk.buildDem(source_data=Metashape.PointCloudData)
        doc.save()
    
        print(" DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    else:
        print("error: transform not available")

    chunk.exportRaster(output_folder + '/dsm'+'.tif', source_data = Metashape.ElevationData)
    
    ground_task = Metashape.Tasks.ClassifyGroundPoints()
    ground_task.apply(chunk)
    chunk.buildDem(source_data=Metashape.PointCloudData, 
               interpolation=Metashape.EnabledInterpolation, 
               classes=[2])  # class 2 = ground points
    
    if chunk.elevation:
        chunk.exportRaster(output_folder + '/dtm'+'.tif', source_data = Metashape.ElevationData)
        print(" DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    
    # create different chunks based on image type
    for ImageType in ImageTypes:
        
        # Duplicate the main chunk
        dup_chunk = chunk.copy()
        dup_chunk.label = ImageType
        doc.chunks.append(dup_chunk)
        print(ImageType+ "Images copied to new chunk "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

        # remove the images from a given chunk
        if ImageType == "MS":
            # Remove RGB cameras from the MS chunk
            rgb_cams = [cam for cam in dup_chunk.cameras if "_D" in cam.label]
            dup_chunk.remove(rgb_cams)
        else:
            # Remove MS cameras from the RGB chunk
            ms_cams = [cam for cam in dup_chunk.cameras if "MS" in cam.label]
            dup_chunk.remove(ms_cams)

        doc.save()
        print(ImageType+ "Images copied to new chunk "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
        #####################################
        # Multispectral calibration
        #####################################
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
            task.apply(dup_chunk)
            
            # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
            dup_chunk.locateReflectancePanels()
            dup_chunk.loadReflectancePanelCalibration(panel_cal_path)
            # Set calibration task
            calib_task = Metashape.Tasks.CalibrateReflectance()
            calib_task.use_reflectance_panels = True
            calib_task.use_sun_sensor = False
            #calib_task.panel_images = calib_cameras
            calib_task.apply(dup_chunk)
            print("multispectral images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
            
            # Find the RGB chunk by label
            dup_chunk.elevation = doc.chunks[1].elevation

        dup_chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
        doc.save()
        
        # export results
        dup_chunk.exportReport(output_folder + '/report'+ImageType+'.pdf')
        
        if dup_chunk.orthomosaic:
            dup_chunk.exportRaster(output_folder + '/orthomosaic'+ImageType+'.tif', source_data = Metashape.OrthomosaicData)

        if dup_chunk.model:
            dup_chunk.exportModel(output_folder + '/model'+ImageType+'.obj')
        
        if dup_chunk.point_cloud:
            dup_chunk.exportPointCloud(output_folder + '/point_cloud'+ImageType+'.las', source_data = Metashape.PointCloudData)

    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()

In [ ]:
# attempt # 4 - one chunk all calibrated together.
# i think this failed...
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]


def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path, ImageTypes = ["RGB","MS"]):
    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')
    chunk = doc.addChunk()
    chunk.label = "ALL"

    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".tif", ".tiff",".jpg", ".jpeg"])  # list of full file paths to your TIFFs
    task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band

    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)
    
    doc.save()
    print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Align images
    ######################################
    # add location accuracies.
    chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
    chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
        
    chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
    doc.save()
    
    chunk.alignCameras()
    doc.save()
    
    print("All images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Optimise alignment
    ######################################
    chunk.optimizeCameras(
        fit_f=True,
        fit_cx=True, fit_cy=True,
        fit_b1=False, fit_b2=False,
        fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
        fit_p1=True, fit_p2=True,
        fit_corrections=False,
        adaptive_fitting=False,
        tiepoint_covariance=False
    )
    print("Images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    #####################################
    # Multispectral calibration
    #####################################
    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
    task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
    task.apply(chunk)
    
    # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
    chunk.locateReflectancePanels()
    chunk.loadReflectancePanelCalibration(panel_cal_path)
    # Set calibration task
    calib_task = Metashape.Tasks.CalibrateReflectance()
    calib_task.use_reflectance_panels = True
    calib_task.use_sun_sensor = False
    #calib_task.panel_images = calib_cameras
    calib_task.apply(chunk)
    print("multispectral images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        

    
    ###############################################
    # Building models and orthos
    ###############################################
    chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
    doc.save()
    
    chunk.buildModel(source_data = Metashape.DepthMapsData)
    doc.save()
    
    chunk.buildUV(page_count = 2, texture_size = 4096)
    doc.save()
    
    chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
    doc.save()
    
    has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
    
    if has_transform:
        chunk.buildPointCloud()
        doc.save()
    
        chunk.buildDem(source_data=Metashape.PointCloudData)
        doc.save()
    
        chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
        doc.save()
    
        print(" ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    else:
        print("error: transform not available")
        
    # export results
    chunk.exportReport(output_folder + '/report'+'.pdf')
    
    if chunk.model:
        chunk.exportModel(output_folder + '/model'+'.obj')
    
    if chunk.point_cloud:
        chunk.exportPointCloud(output_folder + '/point_cloud'+'.las', source_data = Metashape.PointCloudData)
    
    if chunk.elevation:
        chunk.exportRaster(output_folder + '/dsm'+'.tif', source_data = Metashape.ElevationData)
    
    if chunk.orthomosaic:
        chunk.exportRaster(output_folder + '/orthomosaic'+'.tif', source_data = Metashape.OrthomosaicData)
    
    
    ground_task = Metashape.Tasks.ClassifyGroundPoints()
    ground_task.apply(chunk)
    chunk.buildDem(source_data=Metashape.PointCloudData, 
               interpolation=Metashape.EnabledInterpolation, 
               classes=[2])  # class 2 = ground points
    
    if chunk.elevation:
        chunk.exportRaster(output_folder + '/dtm'+'.tif', source_data = Metashape.ElevationData)
        print(" DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()
    

In [ ]:
# Attempt 5 - 2 chunks for MS and RGB
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]


def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path, ImageTypes = ["RGB","MS"]):
    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')

    for ImageType in ImageTypes:
        
        chunk = doc.addChunk()
        chunk.label = ImageType  # Name chunk after image type
        
        # set up photo read in.
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".jpg", ".jpeg",".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
        else:
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to jpgs
        
        task.load_reference = True
        task.load_xmp_accuracy = True
        task.load_xmp_orientation = True
        task.load_xmp_antenna = True
        task.load_xmp_calibration = True
        task.apply(chunk)
        
        doc.save()
        print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
        #####################################
        # Multispectral calibration
        #####################################
        if ImageType == "MS":
            task = Metashape.Tasks.AddPhotos()
            task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
            task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
            task.apply(chunk)
            
            # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
            chunk.locateReflectancePanels()
            chunk.loadReflectancePanelCalibration(panel_cal_path)
            # Set calibration task
            calib_task = Metashape.Tasks.CalibrateReflectance()
            calib_task.use_reflectance_panels = True
            calib_task.use_sun_sensor = False
            #calib_task.panel_images = calib_cameras
            calib_task.apply(chunk)
            print("multispectral images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
            
        ######################################
        # Align images
        ######################################
        # add location accuracies.

        if ImageType == "MS":
            chunk.loadReferenceExif(load_rotation=True, load_accuracy=False)
            #chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
        elif ImageType == "RGB":
            chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
            chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
            
        chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
        doc.save()
        
        chunk.alignCameras()
        doc.save()
        
        print(ImageType + " images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
        ######################################
        # Optimise alignment
        ######################################
        chunk.optimizeCameras(
            fit_f=True,
            fit_cx=True, fit_cy=True,
            fit_b1=False, fit_b2=False,
            fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
            fit_p1=True, fit_p2=True,
            fit_corrections=False,
            adaptive_fitting=False,
            tiepoint_covariance=False
        )
        print(ImageType + " images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

        if ImageType == "MS":
            # After aligning and optimizing MS chunk — now align to RGB
            try:
                chunk_labels = {chunk.label: i for i, chunk in enumerate(doc.chunks)}
                ms_index = chunk_labels.get("MS")
                rgb_index = chunk_labels.get("RGB")
        
                if ms_index is not None and rgb_index is not None:
                    doc.alignChunks(
                        chunks=[ms_index],
                        reference=rgb_index,
                        method=0,  # Point based
                        fit_scale=False,
                        downscale=1,
                        generic_preselection=True,
                        keypoint_limit=40000
                    )
                    print("✅ MS chunk aligned to RGB chunk (before orthos).")
                    doc.save()
                else:
                    print("⚠️ Cannot align chunks: RGB or MS chunk not found.")
            except Exception as e:
                print(f"⚠️ Chunk alignment failed: {e}")
        
        ###############################################
        # Building models and orthos
        ###############################################
        chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
        doc.save()
        
        chunk.buildModel(source_data = Metashape.DepthMapsData)
        doc.save()
        
        chunk.buildUV(page_count = 2, texture_size = 4096)
        doc.save()
        
        chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
        doc.save()
        
        has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
        
        if has_transform:
            chunk.buildPointCloud()
            doc.save()
        
            chunk.buildDem(source_data=Metashape.PointCloudData)
            doc.save()
        
            chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
            doc.save()
        
            print(ImageType + " ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        else:
            print("error: transform not available")
        # export results
        chunk.exportReport(output_folder + '/report'+ImageType+'.pdf')
        
        if chunk.model:
            chunk.exportModel(output_folder + '/model'+ImageType+'.obj')
        
        if chunk.point_cloud:
            chunk.exportPointCloud(output_folder + '/point_cloud'+ImageType+'.las', source_data = Metashape.PointCloudData)
        
        if chunk.elevation:
            chunk.exportRaster(output_folder + '/dsm'+ImageType+'.tif', source_data = Metashape.ElevationData)
        
        if chunk.orthomosaic:
            chunk.exportRaster(output_folder + '/orthomosaic'+ImageType+'.tif', source_data = Metashape.OrthomosaicData)
        
        
        ground_task = Metashape.Tasks.ClassifyGroundPoints()
        ground_task.apply(chunk)
        chunk.buildDem(source_data=Metashape.PointCloudData, 
                   interpolation=Metashape.EnabledInterpolation, 
                   classes=[2])  # class 2 = ground points
        
        
        if chunk.elevation:
            chunk.exportRaster(output_folder + '/dtm'+ImageType+'.tif', source_data = Metashape.ElevationData)
            print(ImageType + " DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()
    

In [ ]:
# Attempt 10: 1 chunk -
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]
        
def ProcessImagesMetashape(doc,image_folder, output_folder, temp_cal_dir, panel_cal_path):

    chunk = doc.addChunk()
    chunk.label = ImageType  # Name chunk after image type
    
    # set up photo read in.
    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".jpg", ".jpeg"])  # list of full file paths to jpgs
    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)
    
    if ImageType == "MS":
        task = Metashape.Tasks.AddPhotos()
        task.filenames = find_files(image_folder, [".tif", ".tiff"])  # list of full file paths to your TIFFs
        task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
        task.load_reference = True
        task.load_xmp_accuracy = True
        task.load_xmp_orientation = True
        task.load_xmp_antenna = True
        task.load_xmp_calibration = True
        task.apply(chunk)

    
    # add location accuracies.
    chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
    chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
    
    doc.save()
    print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
    ######################################
    # Align images
    ######################################
    chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
    doc.save()
    
    chunk.alignCameras()
    doc.save()
    
    print(ImageType+" images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Optimise alignment
    ######################################
    chunk.optimizeCameras(
        fit_f=True,
        fit_cx=True, fit_cy=True,
        fit_b1=False, fit_b2=False,
        fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
        fit_p1=True, fit_p2=True,
        fit_corrections=False,
        adaptive_fitting=False,
        tiepoint_covariance=False
    )
    print(ImageType+" images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    # After aligning and optimizing MS chunk — now align to RGB

    #####################################
    # Multispectral calibration
    #####################################
    if ImageType == "MS":
        task = Metashape.Tasks.AddPhotos()
        task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
        task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
        task.apply(chunk)
        
        # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
        chunk.locateReflectancePanels()
        chunk.loadReflectancePanelCalibration(panel_cal_path)
        # Set calibration task
        calib_task = Metashape.Tasks.CalibrateReflectance()
        calib_task.use_reflectance_panels = True
        calib_task.use_sun_sensor = False
        #calib_task.panel_images = calib_cameras
        calib_task.apply(chunk)
        print(ImageType+" images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    
    try:
        chunk_labels = {chunk.label: i for i, chunk in enumerate(doc.chunks)}
        ms_index = chunk_labels.get("MS")
        rgb_index = chunk_labels.get("RGB")    
    
        if ms_index is not None and rgb_index is not None:
            task = Metashape.Tasks.AlignChunks()
            task.chunks = [ms_index]         # Chunks to align
            task.reference = rgb_index       # Reference chunk index
            task.method = 2                  # 0 = point based, 1 = marker, 2 = camera
            task.fit_scale = False
            task.downscale = 1               # High accuracy (1 = High, 0 = Highest)
            task.generic_preselection = True
            task.keypoint_limit = 40000
            task.apply(doc)
    
            print("✅ MS chunk aligned to RGB chunk (before orthos).")
            doc.save()
        else:
            print("⚠️ Cannot align chunks: RGB or MS chunk not found.")
    except Exception as e:
        print(f"⚠️ Chunk alignment failed: {e}")
        return break
    
    ###############################################
    # Building models and orthos
    ###############################################
    chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
    doc.save()
    
    chunk.buildModel(source_data = Metashape.DepthMapsData)
    doc.save()
    
    chunk.buildUV(page_count = 2, texture_size = 4096)
    doc.save()
    
    chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
    doc.save()
    
    has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
    
    if has_transform:
        chunk.buildPointCloud()
        doc.save()
    
        chunk.buildDem(source_data=Metashape.PointCloudData)
        doc.save()
    
        chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
        doc.save()
    
        print(ImageType+" ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    else:
        print("error: transform not available")

    ###################################
    # export results
    ###################################
    
    chunk.exportReport(output_folder + '/'+ImageType+'report'+'.pdf')
    
    if chunk.model:
        chunk.exportModel(output_folder + '/'+ImageType+'model'+'.obj')
    
    if chunk.point_cloud:
        chunk.exportPointCloud(output_folder + '/'+ImageType+'point_cloud'+'.las', source_data = Metashape.PointCloudData)

    if chunk.orthomosaic and ImageType == "RGB":
        chunk.exportRaster(output_folder + '/'+ImageType+'orthomosaic'+'.tif', source_data = Metashape.OrthomosaicData)
    
    if chunk.orthomosaic and ImageType == "MS":
        task = Metashape.Tasks.ExportRaster()
        task.source_data = Metashape.OrthomosaicData
        task.path = os.path.join(output_folder, f"{ImageType}_orthomosaic.tif")
        task.raster_transform = Metashape.RasterTransformType.RasterTransformNone  # preserve raw bands
        task.save_alpha = False
        task.image_format = Metashape.ImageFormat.ImageFormatTIFF
        task.clip_to_boundary = False  # optional, depending on whether you use shapes
        task.apply(chunk)
        
        for i in range(chunk.orthomosaic.bandCount()):
            chunk.exportRaster(
                path=os.path.join(output_folder, f"MS_band_{i+1}.tif"),
                source_data=Metashape.OrthomosaicData,
                raster_transform=Metashape.RasterTransformType.RasterTransformNone,
                bands=[i]  # zero-based index
            )
    
    if chunk.elevation and ImageType == "RGB":
        # export DSM
        chunk.exportRaster(output_folder + '/dsm'+'.tif', source_data = Metashape.ElevationData)
        
        # classify DTM
        ground_task = Metashape.Tasks.ClassifyGroundPoints()
        ground_task.apply(chunk)
        chunk.buildDem(source_data=Metashape.PointCloudData, 
                   interpolation=Metashape.EnabledInterpolation, 
                   classes=[2])  # class 2 = ground points
        
        # export DTM       
        chunk.exportRaster(output_folder + '/dtm'+'.tif', source_data = Metashape.ElevationData)
        print(" DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()

In [ ]:
# Attempt 8 1 chunk for MS and RGB all corrected
# haven't tried yet...
#####################################
# import required packages
#####################################
import shutil
import pandas as pd
from pathlib import Path
import Metashape
import sys, time
from time import strftime, gmtime
from datetime import datetime, timedelta, timezone

#####################################
# Checking compatibility
#####################################
compatible_major_version = "2.2"
found_major_version = ".".join(Metashape.app.version.split('.')[:2])
if found_major_version != compatible_major_version:
    raise Exception("Incompatible Metashape version: {} != {}".format(found_major_version, compatible_major_version))

#####################################
# Define functions
#####################################

def find_files(folder, extensions):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f)) and os.path.splitext(f)[1].lower() in extensions
    ]


def ProcessImagesMetashape(image_folder, output_folder, temp_cal_dir, panel_cal_path):
    doc = Metashape.Document()
    doc.save(output_folder + '/project.psx')

    chunk = doc.addChunk()
    chunk.label = "ALL"  # Name chunk after image type
    
    # set up photo read in.
    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(image_folder, [".jpg", ".jpeg",".tif", ".tiff"])  # list of full file paths to your TIFFs
    task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band

    
    task.load_reference = True
    task.load_xmp_accuracy = True
    task.load_xmp_orientation = True
    task.load_xmp_antenna = True
    task.load_xmp_calibration = True
    task.apply(chunk)
    
    doc.save()
    print(str(len(chunk.cameras)) + " images loaded "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    #####################################
    # Multispectral calibration
    #####################################
    task = Metashape.Tasks.AddPhotos()
    task.filenames = find_files(temp_cal_dir, [".tif", ".tiff"])  # list of full file paths to your TIFFs
    task.layout = Metashape.MultiplaneLayout  # Multispectral setup: one image per band
    task.apply(chunk)
    
    # calib_cameras = [cam for cam in calib_chunk.cameras] remove "calib_cameras" lines
    chunk.locateReflectancePanels()
    chunk.loadReflectancePanelCalibration(panel_cal_path)
    # Set calibration task
    calib_task = Metashape.Tasks.CalibrateReflectance()
    calib_task.use_reflectance_panels = True
    calib_task.use_sun_sensor = False
    #calib_task.panel_images = calib_cameras
    calib_task.apply(chunk)
    print("multispectral images calibrated "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
        
    ######################################
    # Align images
    ######################################
    # add location accuracies.
    chunk.loadReferenceExif(load_rotation=True, load_accuracy=True)
    chunk.camera_rotation_accuracy = Metashape.Vector([3.0, 2.0, 2.0])  # yaw, pitch, roll
    
    chunk.matchPhotos(keypoint_limit = 40000, tiepoint_limit = 10000, generic_preselection = False, reference_preselection = True)
    doc.save()
    
    chunk.alignCameras()
    doc.save()
    
    print(" images aligned "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    
    ######################################
    # Optimise alignment
    ######################################
    chunk.optimizeCameras(
        fit_f=True,
        fit_cx=True, fit_cy=True,
        fit_b1=False, fit_b2=False,
        fit_k1=True, fit_k2=True, fit_k3=True, fit_k4=False,
        fit_p1=True, fit_p2=True,
        fit_corrections=False,
        adaptive_fitting=False,
        tiepoint_covariance=False
    )
    print(" images optimised "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    # if ImageType == "MS":
    #     # After aligning and optimizing MS chunk — now align to RGB
    #     try:
    #         chunk_labels = {chunk.label: i for i, chunk in enumerate(doc.chunks)}
    #         ms_index = chunk_labels.get("MS")
    #         rgb_index = chunk_labels.get("RGB")
    
    #         if ms_index is not None and rgb_index is not None:
    #             doc.alignChunks(
    #                 chunks=[ms_index],
    #                 reference=rgb_index,
    #                 method=0,  # Point based
    #                 fit_scale=False,
    #                 downscale=1,
    #                 generic_preselection=True,
    #                 keypoint_limit=40000
    #             )
    #             print("✅ MS chunk aligned to RGB chunk (before orthos).")
    #             doc.save()
    #         else:
    #             print("⚠️ Cannot align chunks: RGB or MS chunk not found.")
    #     except Exception as e:
    #         print(f"⚠️ Chunk alignment failed: {e}")
    
    ###############################################
    # Building models and orthos
    ###############################################
    chunk.buildDepthMaps(downscale = 2, filter_mode = Metashape.MildFiltering)
    doc.save()
    
    chunk.buildModel(source_data = Metashape.DepthMapsData)
    doc.save()
    
    chunk.buildUV(page_count = 2, texture_size = 4096)
    doc.save()
    
    chunk.buildTexture(texture_size = 4096, ghosting_filter = True)
    doc.save()
    
    has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation
    
    if has_transform:
        chunk.buildPointCloud()
        doc.save()
    
        chunk.buildDem(source_data=Metashape.PointCloudData)
        doc.save()
    
        chunk.buildOrthomosaic(surface_data=Metashape.ElevationData)
        doc.save()
    
        print(" ortho, DEM, and point cloud created "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    else:
        print("error: transform not available")
    # export results
    chunk.exportReport(output_folder + '/report'+'.pdf')
    
    if chunk.model:
        chunk.exportModel(output_folder + '/model'+'.obj')
    
    if chunk.point_cloud:
        chunk.exportPointCloud(output_folder + '/point_cloud'+'.las', source_data = Metashape.PointCloudData)
    
    if chunk.elevation:
        chunk.exportRaster(output_folder + '/dsm'+'.tif', source_data = Metashape.ElevationData)
    
    if chunk.orthomosaic:
        chunk.exportRaster(output_folder + '/orthomosaic'+'.tif', source_data = Metashape.OrthomosaicData)
    
    
    ground_task = Metashape.Tasks.ClassifyGroundPoints()
    ground_task.apply(chunk)
    chunk.buildDem(source_data=Metashape.PointCloudData, 
               interpolation=Metashape.EnabledInterpolation, 
               classes=[2])  # class 2 = ground points
    
    
    if chunk.elevation:
        chunk.exportRaster(output_folder + '/dtm'+'.tif', source_data = Metashape.ElevationData)
        print(" DTM created and exported "+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))

    print('Processing finished, results saved to ' + output_folder + '.'+ datetime.now(timezone(timedelta(hours=8))).strftime("%H:%M:%S"))
    doc.save()
    Metashape.app.quit()
